## Log-Normal Distribution for Duration Estimation

For duration perception, using a log-normal distribution is much more appropriate because:

1. **Positivity**: Both stimulus durations and observations are naturally positive
2. **Weber's Law**: Noise scales proportionally with duration magnitude
3. **Perceptual Scale**: Duration discrimination follows logarithmic scaling

### Mathematical Framework:
- Let $\log(S)$ be the log-transformed true duration
- Measurements follow: $\log(m) \sim \mathcal{N}(\log(S), \sigma_{log}^2)$
- This means: $m \sim \text{LogNormal}(\log(S), \sigma_{log}^2)$

### Implementation Strategy:
1. Transform durations to log space: $\log(S)$, $\log(m)$
2. Perform all computations in log space (Gaussian operations)
3. Transform back to linear space when needed: $\exp(\log(S))$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import lognorm, norm
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets

# Log-normal implementation for duration estimation
def unimodalMeasurementsLogNormal(sigma_log, S, n=500):
    """
    Generate measurements using log-normal distribution
    Args:
        sigma_log: standard deviation in log space
        S: true duration (in linear space)
        n: number of measurements
    Returns:
        measurements: array of measurements following log-normal distribution
    """
    # Generate log-normal measurements
    # lognorm.rvs(s=sigma_log, scale=S) where s is the shape parameter (sigma in log space)
    measurements = lognorm.rvs(s=sigma_log, scale=S, size=n)
    return measurements

def logNormalPDF(x, S, sigma_log):
    """
    Probability density function for log-normal distribution
    Args:
        x: measurement values
        S: true duration (scale parameterq)
        sigma_log: standard deviation in log space
    """
    return lognorm.pdf(x, s=sigma_log, scale=S)

def likelihoodLogNormal(S, sigma_log):
    """
    Likelihood function for log-normal measurements
    Args:
        S: true duration
        sigma_log: standard deviation in log space
    """
    # Create range from very small positive value to reasonable upper bound
    m = np.linspace(0.01, S + 5*S*sigma_log, 500)
    p_m = logNormalPDF(m, S, sigma_log)
    return m, p_m

def plotLikelihoodLogNormal(S, sigma_log):
    """Plot likelihood function for log-normal distribution"""
    x, p_x = likelihoodLogNormal(S, sigma_log)
    plt.plot(x, p_x, label='Log-Normal Likelihood', linewidth=2)
    plt.xlabel('Measurement Values')
    plt.ylabel('Probability Density')
    plt.title('Log-Normal Measurement Distribution $p(m|S)$')
    plt.legend()

def plotMeasurementsLogNormal(sigma_log, S):
    """Plot histogram of log-normal measurements"""
    m = unimodalMeasurementsLogNormal(sigma_log, S)
    plt.hist(m, bins=50, density=True, alpha=0.5, label='Measurements Histogram')
    plt.xlabel('Measurement Values')
    plt.ylabel('Density')
    plt.title('Log-Normal Measurements Histogram')
    plt.legend()

def plotMeasurementsAndLikelihoodLogNormal(sigma_log, S):
    """Compare measurements histogram with theoretical likelihood"""
    plt.figure(figsize=(12, 6))
    
    # Plot measurements
    plotMeasurementsLogNormal(sigma_log, S)
    
    # Plot likelihood
    plotLikelihoodLogNormal(S, sigma_log)
    
    # Add vertical line for true stimulus
    plt.axvline(S, color='red', linestyle='--', label=f'True Duration S={S:.2f}')
    
    # Add mean of log-normal distribution
    mean_lognormal = S * np.exp(0.5 * sigma_log**2)
    mean_lognormal_pdf = np.mean(unimodalMeasurementsLogNormal(sigma_log, S))
    plt.axvline(mean_lognormal, color='orange', linestyle='--', 
                label=f'Expected Value={mean_lognormal:.2f}')
    plt.axvline(mean_lognormal_pdf, color='purple', linestyle='--', 
                label=f'Mean PDF={mean_lognormal_pdf:.2f}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(0, max(S + 3*S*sigma_log, 2))
    plt.show()

# Interactive comparison between Gaussian and Log-Normal
def compareDistributions(sigma_log, S):
    """Compare Gaussian vs Log-Normal distributions for duration estimation"""
    plt.figure(figsize=(15, 5))
    
    # Gaussian (original approach)
    plt.subplot(1, 3, 1)
    # For Gaussian, use sigma_log as the linear sigma (though this isn't ideal)
    m_gauss = np.random.normal(S, sigma_log * S, 1000)  # Scale sigma with S
    # Remove negative values (hack for Gaussian)
    m_gauss = m_gauss[m_gauss > 0]
    plt.hist(m_gauss, bins=30, density=True, alpha=0.7, color='blue', label='Gaussian')
    plt.axvline(S, color='red', linestyle='--', label=f'True S={S:.2f}')
    plt.title('Gaussian Distribution\n(with negative values removed)')
    plt.xlabel('Measurement')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Log-Normal (proposed approach)
    plt.subplot(1, 3, 2)
    m_lognorm = unimodalMeasurementsLogNormal(sigma_log, S, 1000)
    plt.hist(m_lognorm, bins=30, density=True, alpha=0.7, color='green', label='Log-Normal')
    plt.axvline(S, color='red', linestyle='--', label=f'True S={S:.2f}')
    mean_lognormal = S * np.exp(0.5 * sigma_log**2)
    plt.axvline(mean_lognormal, color='orange', linestyle='--', 
                label=f'E[m]={mean_lognormal:.2f}')
    plt.title('Log-Normal Distribution\n(naturally positive)')
    plt.xlabel('Measurement')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Weber's Law demonstration
    plt.subplot(1, 3, 3)
    S_values = np.linspace(0.1, 2.0, 10)
    cv_lognormal = []  # Coefficient of variation
    
    for S_val in S_values:
        measurements = unimodalMeasurementsLogNormal(sigma_log, S_val, 1000)
        cv = np.std(measurements) / np.mean(measurements)
        cv_lognormal.append(cv)
    
    plt.plot(S_values, cv_lognormal, 'o-', color='green', label='Log-Normal CV')
    plt.axhline(sigma_log, color='red', linestyle='--', 
                label=f'σ_log = {sigma_log:.2f}')
    plt.title("Weber's Law: CV vs Duration")
    plt.xlabel('True Duration S')
    plt.ylabel('Coefficient of Variation')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Interactive widgets
print("Log-Normal Duration Estimation Implementation")
print("=" * 50)

interact(plotMeasurementsAndLikelihoodLogNormal,
         sigma_log=FloatSlider(value=0.2, min=0.05, max=0.8, step=0.05, 
                               description='σ_log (noise in log space)'),
         S=FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, 
                       description='True Duration S'))

print("\nComparison: Gaussian vs Log-Normal")
print("=" * 40)

interact(compareDistributions,
         sigma_log=FloatSlider(value=0.3, min=0.1, max=0.6, step=0.05, 
                               description='σ_log'),
         S=FloatSlider(value=0.8, min=0.2, max=2.0, step=0.2, 
                       description='True Duration S'))

Log-Normal Duration Estimation Implementation


interactive(children=(FloatSlider(value=0.2, description='σ_log (noise in log space)', max=0.8, min=0.05, step…


Comparison: Gaussian vs Log-Normal


interactive(children=(FloatSlider(value=0.3, description='σ_log', max=0.6, min=0.1, step=0.05), FloatSlider(va…

<function __main__.compareDistributions(sigma_log, S)>

In [ ]:
from scipy import stats
def plot_lognormal_perspectives(true_S=0.5, sigma_log=0.4, n_measurements=5):
    plt.figure(figsize=(12, 8))
    
    # For lognormal, we need to convert parameters
    # If X is lognormal, then log(X) is normal with mean mu and std sigma_log
    mu_log = np.log(true_S) - sigma_log**2/2  # Adjustment to make the mean of lognormal = true_S
    
    # 1. Experimenter's perspective: Measurement distribution p(m|s)
    m_range = np.linspace(0.01, true_S*3, 1000)  # Avoid m=0 for lognormal
    measurement_dist = stats.lognorm.pdf(m_range, sigma_log, scale=np.exp(mu_log))
    
    plt.subplot(2, 1, 1)
    plt.plot(m_range, measurement_dist, 'b-', linewidth=2, 
             label=f'Measurement Distribution p(m|s={true_S})')
    plt.fill_between(m_range, measurement_dist, alpha=0.2, color='blue')
    plt.xlabel('Possible measurements (m)')
    plt.ylabel('Probability density')
    plt.title("Experimenter's Perspective: Lognormal Measurement Distribution")
    
    # Generate some example measurements
    np.random.seed(42)  # For reproducibility
    measurements = np.random.lognormal(mean=mu_log, sigma=sigma_log, size=n_measurements)
    
    # Add measurements as vertical lines
    for i, m in enumerate(measurements):
        plt.axvline(m, color=f'C{i+1}', linestyle='--', 
                    label=f'Measurement {i+1}: m={m:.2f}')
    plt.legend()
    
    # 2. Observer's perspective: Likelihood functions L(s|m)
    plt.subplot(2, 1, 2)
    s_range = np.linspace(0.01, true_S*3, 1000)  # Avoid s=0 for lognormal
    
    # Plot likelihood functions for each measurement
    for i, m in enumerate(measurements):
        likelihoods = []
        for s in s_range:
            # For each possible stimulus value s, calculate mu_log for that s
            s_mu_log = np.log(s) - sigma_log**2/2
            # Likelihood is proportional to p(m|s)
            likelihood = stats.lognorm.pdf(m, sigma_log, scale=np.exp(s_mu_log))

            likelihoods.append(likelihood)
        
        # Normalize for better visualization
        likelihoods = np.array(likelihoods) / np.max(likelihoods)
        
        plt.plot(s_range, likelihoods, color=f'C{i+1}', 
                 label=f'Likelihood L(s|m={m:.2f})')
        plt.axvline(m, color=f'C{i+1}', linestyle='--', 
                    label=f'Measurement {i+1}: m={m:.2f}')

    #print(f"Likelihood for measurement {i+1} (m={m:.2f}): {likelihoods[:100]}...")

    plt.axvline(true_S, color='k', linestyle='-', label='True stimulus')
    plt.xlabel('Possible stimulus values (s)')
    plt.ylabel('Likelihood (normalized)')
    plt.title("Observer's Perspective: Likelihood Functions")
    plt.legend()
    
    plt.tight_layout()
    plt.show()

interact(plot_lognormal_perspectives,
         true_S=FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, 
                            description='True Duration S'),
         sigma_log=FloatSlider(value=0.4, min=0.1, max=1.0, step=0.1, 
                               description='σ_log (noise in log space)'),
         n_measurements=widgets.IntSlider(value=1, min=1, max=10, step=1, 
                                          description='Number of Measurements'))

interactive(children=(FloatSlider(value=0.5, description='True Duration S', max=2.0, min=0.1), FloatSlider(val…

<function __main__.plot_lognormal_perspectives(true_S=0.5, sigma_log=0.4, n_measurements=5)>

## Important Distinction: Experimenter vs Observer Perspectives

### **Experimenter's Perspective**: Measurement Distribution p(m|s)
- **What it shows**: How measurements are distributed when the experimenter presents a specific stimulus S
- **The process**: Stimulus S → Perceptual system (with noise) → Measurement m
- **Distribution**: m ~ LogNormal(log(S), σ_log²) 
- **Key insight**: The log-normal distribution describes the **measurement noise**, not the stimulus generation

### **Observer's Perspective**: Likelihood Function L(s|m)  
- **What it shows**: Given a measurement m, how likely is each possible stimulus value s?
- **The process**: Observed measurement m → Infer possible stimulus values s
- **Mathematical relationship**: L(s|m) ∝ p(m|s)
- **Key insight**: This is Bayesian inference - using the measurement to estimate the stimulus

### **Why Log-Normal for Measurements?**
1. **Weber's Law**: Perceptual noise scales with stimulus magnitude
2. **Positivity**: Both stimuli and measurements must be positive (durations > 0)
3. **Empirical evidence**: Duration perception shows multiplicative noise

### **Correct Interpretation**:
- **NOT**: "Stimuli are log-normally distributed in the experiment"
- **YES**: "Measurements of stimuli are log-normally distributed due to perceptual noise"

The experimenter presents discrete stimulus values, but the observer's perceptual system adds log-normal noise to create the measurement distribution.

In [3]:
# Causal Inference with Log-Normal Distributions

def bimodalMeasurementsLogNormal(sigma_A_log, sigma_V_log, S, n=500):
    """
    Generate bimodal measurements using log-normal distributions
    Args:
        sigma_A_log: auditory noise in log space
        sigma_V_log: visual noise in log space
        S: true duration
        n: number of measurements
    """
    # Auditory measurements
    mA = lognorm.rvs(s=sigma_A_log, scale=S, size=n)
    
    # Visual measurements
    mV = lognorm.rvs(s=sigma_V_log, scale=S, size=n)
    
    return mA, mV

def causalInferenceLogNormal(mA, mV, sigma_A_log, sigma_V_log, pC):
    """
    Causal inference with log-normal measurements
    Working in log space for computations
    """
    # Convert measurements to log space
    log_mA = np.log(mA)
    log_mV = np.log(mV)
    
    # Prior parameters in log space (assume log(S) ~ N(mu_prior, sigma_prior^2))
    mu_prior_log = 0  # Assuming geometric mean of 1 (log(1) = 0)
    sigma_prior_log = 1.0  # Prior uncertainty in log space
    
    # Compute posterior for C=1 (common cause) in log space
    # Combined measurement: weighted average in log space
    J_a = 1 / sigma_A_log**2
    J_v = 1 / sigma_V_log**2
    J_p = 1 / sigma_prior_log**2
    
    # Posterior precision and mean for common cause (in log space)
    precision_C1 = J_a + J_v + J_p
    mu_C1_log = (J_a * log_mA + J_v * log_mV + 
                 J_p * mu_prior_log) / precision_C1
    sigma_C1_log = 1 / np.sqrt(precision_C1)
    
    # Posterior for C=0 (separate causes) in log space
    # Auditory estimate
    J_a_post = J_a + J_p
    mu_A_post_log = (J_a * log_mA + J_p * mu_prior_log) / J_a_post
    sigma_A_post_log = 1 / np.sqrt(J_a_post)
    
    # Visual estimate  
    J_v_post = J_v + J_p
    mu_V_post_log = (J_v * log_mV + J_p * mu_prior_log) / J_v_post
    sigma_V_post_log = 1 / np.sqrt(J_v_post)
    
    # Likelihood of measurements under each causal structure
    # P(mA, mV | C=1): joint likelihood in log space
    diff_log = log_mA - log_mV
    sigma_diff_log = np.sqrt(sigma_A_log**2 + sigma_V_log**2)
    likelihood_C1 = norm.pdf(diff_log, 0, sigma_diff_log)
    
    # P(mA, mV | C=0): independent likelihoods
    likelihood_C0 = 1  # Normalized constant (doesn't affect relative probabilities)
    
    # Posterior probability of common cause
    evidence_C1 = likelihood_C1 * pC
    evidence_C0 = likelihood_C0 * (1 - pC)
    pC_post = evidence_C1 / (evidence_C1 + evidence_C0)
    
    # Final estimates (convert back to linear space)
    if pC_post > 0.5:
        # Common cause: use integrated estimate
        S_hat = np.exp(mu_C1_log)
        causal_structure = 'Common'
    else:
        # Separate causes: use modality-specific estimates
        S_hat_A = np.exp(mu_A_post_log)
        S_hat_V = np.exp(mu_V_post_log)
        # For demonstration, return auditory estimate
        S_hat = S_hat_A
        causal_structure = 'Separate'
    
    return S_hat, pC_post, causal_structure

def simulateCausalInferenceLogNormal(sigma_A_log=0.3, sigma_V_log=0.25, 
                                   S_true=0.8, pC_prior=0.7, n_trials=1000):
    """
    Simulate causal inference experiment with log-normal measurements
    """
    # Generate measurements
    mA, mV = bimodalMeasurementsLogNormal(sigma_A_log, sigma_V_log, S_true, n_trials)
    
    # Perform causal inference for each trial
    S_estimates = []
    pC_posteriors = []
    causal_decisions = []
    
    for i in range(n_trials):
        S_hat, pC_post, decision = causalInferenceLogNormal(
            mA[i], mV[i], sigma_A_log, sigma_V_log, pC_prior)
        
        S_estimates.append(S_hat)
        pC_posteriors.append(pC_post)
        causal_decisions.append(decision)
    
    return np.array(S_estimates), np.array(pC_posteriors), causal_decisions, mA, mV

def plotCausalInferenceResultsLogNormal(sigma_A_log=0.3, sigma_V_log=0.25, 
                                       S_true=0.8, pC_prior=0.7):
    """
    Plot results of causal inference simulation with log-normal distributions
    """
    S_estimates, pC_posteriors, decisions, mA, mV = simulateCausalInferenceLogNormal(
        sigma_A_log, sigma_V_log, S_true, pC_prior, 1000)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # Raw measurements
    axes[0, 0].scatter(mA[:100], mV[:100], alpha=0.6, s=20)
    axes[0, 0].plot([0, max(max(mA), max(mV))], [0, max(max(mA), max(mV))], 
                    'r--', alpha=0.7, label='Unity line')
    axes[0, 0].axvline(S_true, color='red', linestyle=':', label=f'True S={S_true}')
    axes[0, 0].axhline(S_true, color='red', linestyle=':', alpha=0.7)
    axes[0, 0].set_xlabel('Auditory Measurement')
    axes[0, 0].set_ylabel('Visual Measurement')
    axes[0, 0].set_title('Raw Measurements (Log-Normal)')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Posterior probabilities
    axes[0, 1].hist(pC_posteriors, bins=50, alpha=0.7, density=True)
    axes[0, 1].axvline(pC_prior, color='red', linestyle='--', 
                       label=f'Prior p(C=1)={pC_prior}')
    axes[0, 1].axvline(np.mean(pC_posteriors), color='orange', linestyle='--',
                       label=f'Mean Posterior={np.mean(pC_posteriors):.3f}')
    axes[0, 1].set_xlabel('Posterior p(C=1|mA,mV)')
    axes[0, 1].set_ylabel('Density')
    axes[0, 1].set_title('Causal Inference Posteriors')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Duration estimates
    axes[0, 2].hist(S_estimates, bins=50, alpha=0.7, density=True, 
                    label='Estimates')
    axes[0, 2].axvline(S_true, color='red', linestyle='--', 
                       label=f'True Duration={S_true}')
    axes[0, 2].axvline(np.mean(S_estimates), color='orange', linestyle='--',
                       label=f'Mean Estimate={np.mean(S_estimates):.3f}')
    axes[0, 2].set_xlabel('Duration Estimate')
    axes[0, 2].set_ylabel('Density')
    axes[0, 2].set_title('Final Duration Estimates')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # Causal decisions over time
    common_indices = [i for i, d in enumerate(decisions) if d == 'Common']
    separate_indices = [i for i, d in enumerate(decisions) if d == 'Separate']
    
    axes[1, 0].scatter(common_indices[:50], [S_estimates[i] for i in common_indices[:50]], 
                       alpha=0.7, color='blue', label='Common Cause', s=30)
    axes[1, 0].scatter(separate_indices[:50], [S_estimates[i] for i in separate_indices[:50]], 
                       alpha=0.7, color='red', label='Separate Causes', s=30)
    axes[1, 0].axhline(S_true, color='black', linestyle='--', alpha=0.7, 
                       label=f'True Duration={S_true}')
    axes[1, 0].set_xlabel('Trial Number')
    axes[1, 0].set_ylabel('Duration Estimate')
    axes[1, 0].set_title('Estimates by Causal Decision')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Error analysis
    errors = S_estimates - S_true
    axes[1, 1].hist(errors, bins=50, alpha=0.7, density=True)
    axes[1, 1].axvline(0, color='red', linestyle='--', label='No error')
    axes[1, 1].axvline(np.mean(errors), color='orange', linestyle='--',
                       label=f'Mean Error={np.mean(errors):.3f}')
    axes[1, 1].set_xlabel('Estimation Error')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].set_title('Error Distribution')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # Weber's Law analysis
    S_values = np.linspace(0.2, 2.0, 10)
    cv_values = []
    
    for S_val in S_values:
        estimates_temp, _, _, _, _ = simulateCausalInferenceLogNormal(
            sigma_A_log, sigma_V_log, S_val, pC_prior, 200)
        cv = np.std(estimates_temp) / np.mean(estimates_temp)
        cv_values.append(cv)
    
    axes[1, 2].plot(S_values, cv_values, 'o-', color='green', linewidth=2)
    axes[1, 2].set_xlabel('True Duration')
    axes[1, 2].set_ylabel('Coefficient of Variation')
    axes[1, 2].set_title("Weber's Law: CV vs Duration")
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print(f"Summary Statistics (Log-Normal Model):")
    print(f"True Duration: {S_true:.3f}")
    print(f"Mean Estimate: {np.mean(S_estimates):.3f} ± {np.std(S_estimates):.3f}")
    print(f"Bias: {np.mean(errors):.3f}")
    print(f"RMSE: {np.sqrt(np.mean(errors**2)):.3f}")
    print(f"Proportion Common Cause Decisions: {len(common_indices)/len(decisions):.3f}")

# Interactive exploration
print("\nCausal Inference with Log-Normal Distributions")
print("=" * 50)

interact(plotCausalInferenceResultsLogNormal,
         sigma_A_log=FloatSlider(value=0.3, min=0.1, max=0.6, step=0.05, 
                                description='σ_A (log)'),
         sigma_V_log=FloatSlider(value=0.25, min=0.1, max=0.6, step=0.05, 
                                description='σ_V (log)'),
         S_true=FloatSlider(value=0.8, min=0.2, max=2.0, step=0.1, 
                           description='True Duration'),
         pC_prior=FloatSlider(value=0.7, min=0.1, max=0.9, step=0.1, 
                             description='Prior p(C=1)'))


Causal Inference with Log-Normal Distributions


interactive(children=(FloatSlider(value=0.3, description='σ_A (log)', max=0.6, min=0.1, step=0.05), FloatSlide…

<function __main__.plotCausalInferenceResultsLogNormal(sigma_A_log=0.3, sigma_V_log=0.25, S_true=0.8, pC_prior=0.7)>

## Key Advantages of Log-Normal Approach

### 1. **Theoretical Justification**
- **Weber's Law**: Discrimination thresholds scale with stimulus magnitude
- **Natural Positivity**: No need for ad-hoc constraints to prevent negative durations
- **Perceptual Scaling**: Matches logarithmic nature of duration perception

### 2. **Mathematical Benefits**
- **Gaussian Operations in Log Space**: All computations use familiar Gaussian math
- **Multiplicative Noise**: More realistic for duration perception
- **Stable Parameter Estimation**: Better numerical properties

### 3. **Empirical Support**
- Duration discrimination follows Weber's law
- Reaction time distributions are typically log-normal
- Neural timing mechanisms suggest multiplicative noise

### 4. **Implementation Advantages**
- **Clean Bayesian Updates**: Standard Gaussian inference in log space
- **No Boundary Issues**: Natural positivity constraint
- **Scalable Noise**: Automatically handles magnitude-dependent uncertainty

### Key Transformation:
```
Original: m ~ N(S, σ²)        # Can be negative, constant absolute noise
Log-Normal: log(m) ~ N(log(S), σ_log²)  # Always positive, relative noise
```

This approach naturally implements Weber's law where noise scales with duration magnitude!